In [8]:
from keras import models
from keras import layers
from keras import optimizers
from keras.datasets import imdb 
import numpy as np

## 1. Splitting the data into three types(training, validation, test)

### 1) Hold out validation example
*One big disadvantage of this validation arises esp when we don't have many data points enough for validation& test data to statistically represent the whole data.  
*This problem can be detected if the model performance substantially differs when we re-partition the data.  
*K-fold cross validation, Iterated K-fold cross-validation will be good remedies for this problem.

In [10]:
num_validation_samples=10000
np.random.shuffle(data) #it's 'usually' better to first shuffle the data

validation_data=data[:num_validation_samples] # create validation dataset
data=data[num_validation_samples:]
trainindg_data=data[:] #create training dataset

#Train the model w/ training set and evaluate the model w/ validation set
#Model tuning repeatedly occurs in this stage
model=get_model()
model.train(training_data)
validation_score=model.evaluate(validation_data)

#after the hyper-parameter tuning, we re-trainthe model using all data except test data
model=get_model()
model.train(np.concatenate([training_data,validation_data]))
test_score=model.evaludate(test_data) #calculate the final score of interest

NameError: name 'data' is not defined

### 2) K-cross validation

In [ ]:
#4-fold cross validation
k=4
num_validation_samples=len(data)//k

np.random.shuffle(data)
validation_scores=[]
for fold in range(k):
    validation_data=data[num_validation_samples*fold: num_validation_samples*(fold+1)]
    training_data=data[:num_validation_samples*fold]+
    data[num_validation_samples*(fold+1):] #Linking two lists
    
    model=get_model() #creating a new, untrained model
    
    #Train the model w/ training set and evaluate the model w/ validation set
    #Model tuning repeatedly occurs in this stage
    model.train(training_data)
    validation_score=model.evaludate(validation_data)
    validation_scores.append(validation_score)
    validation_score=np.average(validation_score) #validation score: average of K-fold validation scores

#after the hyper-parameter tuning, we re-trainthe model using all data except test data
model=get_model()
model.train(data)
test_score=model.evaluate(test_data) #calculate the final score of interest
                      

## 2. Data Preparation
1)Vectorization: transforming various type of data into tensors  
2)Normalization: making different features follow uniform distribution, usually N(0,1)  
3)Dealing with omitted values: if test data inclues ommited values, then purposely omit some values in training data.  
4)Feature Engineering: utilizing background knowledge to express features in a simple way to solve problem more easily.

## 3. Overfitting & Underfitting
 *The fundamental issue of machine learning is finding the balance between optimization and generalization.  
 *When optimization and generalization improve at the same time, the model is underfitted, and has room for improvement.  
 *When optimization gets better at the expense of generalization, the model starts to be overfitted.  
 *The best policty to prevent overfitting (=improve generalization) is to gather more data.  
 *When gathering more data is not feasible, the second best policy is limiting the model's capacity to absorb or store information, which is called 'regularization'.

### 3.1) Reducing the size of Network  
*You need to manually increase/decrease to find the optimal number of layers and units.  
*While more layers & units ensure fast learning, it also makes the model more vulnerable to overfitting.

### 3.2) Weight Regularization

In [14]:
from keras import regularizers

# L2 regularize: adding penalty term to the 'training' loss
model=models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), 
                       activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), 
                       activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#You can either use L1 regularization, or L1& L2 regularization.
## regularizers.l1(0.001)
## regularizers.l1_l2(l1=0.001, l2=0.001)

### 3.3) Dropout

![](ch4_1.png)

In [16]:
## Adding two Dropoutlayers and see how wellit prevents overfitting.

model=models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5)) #1
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5)) #2
model.add(layers.Dense(1, activation='sigmoid'))